In [2]:
# =============================================================
# Copyright © 2020 Intel Corporation
# 
# SPDX-License-Identifier: MIT
# =============================================================

# Modin Getting Started Example for Distributed Pandas

## Importing and Organizing Data

In this example we will be generating a **synthetic dataset** and **demonstrating stock Pandas operations running with Modin**.

Let's start by **importing** all the necessary packages and modules

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import time

In [4]:
# ****** Do not change the code in this cell! It verifies that the notebook is being run correctly! ******

def verify_and_print_times(pandas_time, modin_time):
    if modin_time < pandas_time:
        print(f"Modin was {pandas_time / modin_time:.2f}X faster than stock pandas!")
        return
    print(
        f"Oops, stock pandas appears to be {modin_time / pandas_time:.2f}X faster than Modin in this case. "
        "This is unlikely but could happen sometimes on certain machines/environments/datasets. "
        "One of the most probable reasons is the excessive amount of partitions being assigned to a single worker. "
        "You may visit Modin's optimization guide in order to learn more about such cases and how to fix them: "
        "\nhttps://modin.readthedocs.io/en/latest/usage_guide/optimization_notes/index.html\n\n"
        "But first, verify that you're using the latest Modin version, also, try to use different executions, "
        "for basic usage we recommend non-experimental 'PandasOnRay'.\n"
        "Current configuration is:"
    )
    try:
        from modin.utils import get_current_execution

        execution = get_current_execution()
    except ImportError:
        # for modin version < 0.12.0
        try:
            from modin.utils import get_current_backend

            execution = get_current_backend()
        except ImportError:
            # for modin versions < 0.8.1
            execution = (
                "Can't deduce the current execution, your Modin version is too old!"
            )
    print(f"\tExecution: {execution}")
    try:
        import modin.config as cfg

        print(
            f"\tIs experimental: {cfg.IsExperimental.get()}\n"
            f"\tNumber of CPUs to utilize by Modin (check that Modin uses all CPUs on your machine): {cfg.CpuCount.get()}\n"
            f"\tIs in debug mode (debug mode may perform slower): {cfg.IsDebug.get()}"
        )
    except (ImportError, AttributeError):
        # for modin versions < 0.8.2
        print("\tCan't deduce Modin configuration, your Modin version is too old!")
    import modin

    print(f"\tModin version: {modin.__version__}")

## How to Use Modin

We will also be importing **stock Pandas as pandas** and **Modin as pd to show differentiation**. You can see importing Modin is simple and **does not require any additional steps.**

In [5]:
import pandas

In [6]:
import modin.pandas as pd

We will now **generate a synthetic dataset** using NumPy to use with Modin and save it to a CSV.

In [7]:
array=np.random.randint(low=100,high=10000,size=(2**18,2**8))
#array
np.savetxt("foo.csv", array, delimiter=",") #how to generate array

Now we will convert the ndarray into a Pandas dataframe and display the first five rows.
For **stock pandas, the dataframe is being stored as `pandas_df`** and for **Modin, the same dataframe is being stored as `modin_df`**.
Let's try running the following cell with Pandas first.

In [8]:
%%time
pandas_df = pandas.read_csv("foo.csv", names=["col{}".format(i) for i in range(256)])
pandas_df.head()

CPU times: user 12 s, sys: 1.6 s, total: 13.6 s
Wall time: 15.5 s


,col0,col1,col2,col3,col4,col5,col6,col7,col8,col9,...,col246,col247,col248,col249,col250,col251,col252,col253,col254,col255
0,3403.0,7232.0,8592.0,6451.0,5741.0,9222.0,9565.0,1560.0,8212.0,7245.0,...,801.0,8949.0,6749.0,5436.0,9975.0,7339.0,3643.0,1690.0,3314.0,3417.0
1,8905.0,7830.0,1765.0,9886.0,3208.0,7536.0,9996.0,1495.0,607.0,7374.0,...,7688.0,6998.0,6585.0,8522.0,4443.0,2206.0,3721.0,4717.0,509.0,8320.0
2,8039.0,1927.0,9661.0,7323.0,386.0,2187.0,1262.0,7503.0,4353.0,1610.0,...,5627.0,6633.0,5627.0,8256.0,3143.0,7739.0,946.0,6071.0,1431.0,1575.0
3,7142.0,9296.0,3865.0,6093.0,2408.0,7508.0,177.0,2156.0,5027.0,5917.0,...,1101.0,3271.0,6345.0,539.0,5378.0,5209.0,7718.0,3773.0,9630.0,8022.0
4,2555.0,1853.0,8666.0,3898.0,2909.0,9641.0,3251.0,1531.0,3485.0,4762.0,...,5797.0,2574.0,844.0,9498.0,4877.0,7506.0,3134.0,5152.0,4078.0,9271.0


Now let's run the same code, but use **Modin instead of stock Pandas.**

**Note the speedup!**

In [9]:
%%time
modin_df=pd.read_csv("foo.csv", names=["col{}".format(i) for i in range(256)])
modin_df.head()

To remove this warning, run the following python code before doing dataframe operations:

    import ray
    ray.init()



CPU times: user 2.62 s, sys: 784 ms, total: 3.4 s
Wall time: 9.27 s


,col0,col1,col2,col3,col4,col5,col6,col7,col8,col9,...,col246,col247,col248,col249,col250,col251,col252,col253,col254,col255
0,3403.0,7232.0,8592.0,6451.0,5741.0,9222.0,9565.0,1560.0,8212.0,7245.0,...,801.0,8949.0,6749.0,5436.0,9975.0,7339.0,3643.0,1690.0,3314.0,3417.0
1,8905.0,7830.0,1765.0,9886.0,3208.0,7536.0,9996.0,1495.0,607.0,7374.0,...,7688.0,6998.0,6585.0,8522.0,4443.0,2206.0,3721.0,4717.0,509.0,8320.0
2,8039.0,1927.0,9661.0,7323.0,386.0,2187.0,1262.0,7503.0,4353.0,1610.0,...,5627.0,6633.0,5627.0,8256.0,3143.0,7739.0,946.0,6071.0,1431.0,1575.0
3,7142.0,9296.0,3865.0,6093.0,2408.0,7508.0,177.0,2156.0,5027.0,5917.0,...,1101.0,3271.0,6345.0,539.0,5378.0,5209.0,7718.0,3773.0,9630.0,8022.0
4,2555.0,1853.0,8666.0,3898.0,2909.0,9641.0,3251.0,1531.0,3485.0,4762.0,...,5797.0,2574.0,844.0,9498.0,4877.0,7506.0,3134.0,5152.0,4078.0,9271.0


Let's now **visualize** this speedup from Modin with a plot!

In [10]:
def plotter(outputdict):
    fig = plt.figure(figsize = (10, 5)) 
    plt.bar(outputdict.keys(),outputdict.values(),color='blue',width=0.4)
    plt.xlabel("Python Package")
    plt.ylabel("Runtime(seconds)")
    plt.show()

Pandas Time(seconds): 14.495349168777466 
Modin Time(seconds): 2.903475761413574
Modin was 4.99X faster than stock pandas!


In [11]:
t0 = time.time()
pandas_df = pandas.read_csv("foo.csv", names=["col{}".format(i) for i in range(256)])
pandas_time = time.time()- t0

t1 = time.time()
modin_df = pd.read_csv("foo.csv", names=["col{}".format(i) for i in range(256)])
modin_time = time.time() - t1

print("Pandas Time(seconds):",pandas_time,"\nModin Time(seconds):",modin_time)
verify_and_print_times(pandas_time, modin_time)
outputDict={"Pandas":pandas_time,"Modin":modin_time}
plotter(outputDict)

## Other DataFrame Function Performance Example
We will now show the speedup in performance from Modin compared to stock Pandas with a few common functions.

Like before, **`pandas_df` is for  stock Pandas**, **`modin_df` is for Modin**.

### `df.mean()`

In [12]:
# Mean
t2 = time.time()
pandas_df.mean(axis=0)
pandas_time=time.time()- t2
print(" stock Pandas wall time for completion in seconds:",pandas_time)

 stock Pandas wall time for completion in seconds: 0.10071349143981934


In [13]:
# Mean
t3 = time.time()
modin_df.mean(axis=0)
modin_time=time.time()- t3
print("Modin wall time for completion in seconds:",modin_time)

Modin wall time for completion in seconds: 0.4911317825317383


In [14]:
verify_and_print_times(pandas_time, modin_time)

Oops, stock pandas appears to be 4.88X faster than Modin in this case. This is unlikely but could happen sometimes on certain machines/environments/datasets. One of the most probable reasons is the excessive amount of partitions being assigned to a single worker. You may visit Modin's optimization guide in order to learn more about such cases and how to fix them: 
https://modin.readthedocs.io/en/latest/usage_guide/optimization_notes/index.html

But first, verify that you're using the latest Modin version, also, try to use different executions, for basic usage we recommend non-experimental 'PandasOnRay'.
Current configuration is:
	Execution: PandasOnRay
	Is experimental: None
	Number of CPUs to utilize by Modin (check that Modin uses all CPUs on your machine): 96
	Is in debug mode (debug mode may perform slower): None
	Modin version: 0.11.3


### `df.applymap`

In [15]:
# Long apply function
t6 = time.time()
print(pandas_df.applymap(lambda x: x + 1))
pandas_time = time.time() - t6
print(" stock Pandas wall time for completion in seconds:",pandas_time)

          col0    col1    col2    col3    col4    col5    col6    col7  \
0       3404.0  7233.0  8593.0  6452.0  5742.0  9223.0  9566.0  1561.0   
1       8906.0  7831.0  1766.0  9887.0  3209.0  7537.0  9997.0  1496.0   
2       8040.0  1928.0  9662.0  7324.0   387.0  2188.0  1263.0  7504.0   
3       7143.0  9297.0  3866.0  6094.0  2409.0  7509.0   178.0  2157.0   
4       2556.0  1854.0  8667.0  3899.0  2910.0  9642.0  3252.0  1532.0   
...        ...     ...     ...     ...     ...     ...     ...     ...   
262139  6901.0  7773.0  1994.0  6078.0  8446.0  9981.0  6879.0   720.0   
262140  1888.0  5857.0  7019.0  9730.0  8134.0  2428.0  1942.0  2338.0   
262141   296.0  5778.0  7250.0  4353.0  4532.0  6823.0  2552.0  4964.0   
262142  9087.0  3299.0  9104.0  1698.0  3116.0  9883.0  5172.0  4198.0   
262143   946.0  2234.0  4352.0   975.0  8170.0  7792.0  9190.0  8470.0   

          col8    col9  ...  col246  col247  col248  col249  col250  col251  \
0       8213.0  7246.0  ...   80

In [16]:
# Long apply function
t7 = time.time()
print(modin_df.applymap(lambda x: x + 1))
modin_time = time.time() - t7
print("Modin wall time for completion in seconds:",modin_time)

          col0    col1    col2    col3    col4    col5    col6    col7  \
0       3404.0  7233.0  8593.0  6452.0  5742.0  9223.0  9566.0  1561.0   
1       8906.0  7831.0  1766.0  9887.0  3209.0  7537.0  9997.0  1496.0   
2       8040.0  1928.0  9662.0  7324.0   387.0  2188.0  1263.0  7504.0   
3       7143.0  9297.0  3866.0  6094.0  2409.0  7509.0   178.0  2157.0   
4       2556.0  1854.0  8667.0  3899.0  2910.0  9642.0  3252.0  1532.0   
...        ...     ...     ...     ...     ...     ...     ...     ...   
262139  6901.0  7773.0  1994.0  6078.0  8446.0  9981.0  6879.0   720.0   
262140  1888.0  5857.0  7019.0  9730.0  8134.0  2428.0  1942.0  2338.0   
262141   296.0  5778.0  7250.0  4353.0  4532.0  6823.0  2552.0  4964.0   
262142  9087.0  3299.0  9104.0  1698.0  3116.0  9883.0  5172.0  4198.0   
262143   946.0  2234.0  4352.0   975.0  8170.0  7792.0  9190.0  8470.0   

          col8    col9  ...  col246  col247  col248  col249  col250  col251  \
0       8213.0  7246.0  ...   80

In [17]:
verify_and_print_times(pandas_time, modin_time)

Modin was 23.31X faster than stock pandas!


### `pd.concat([df, df])`

In [18]:
# Concat
t8 = time.time()
print(pandas.concat([pandas_df, pandas_df], axis=0))
pandas_time = time.time() - t8
print("stock Pandas wall time for completion in seconds:",pandas_time)

          col0    col1    col2    col3    col4    col5    col6    col7  \
0       3403.0  7232.0  8592.0  6451.0  5741.0  9222.0  9565.0  1560.0   
1       8905.0  7830.0  1765.0  9886.0  3208.0  7536.0  9996.0  1495.0   
2       8039.0  1927.0  9661.0  7323.0   386.0  2187.0  1262.0  7503.0   
3       7142.0  9296.0  3865.0  6093.0  2408.0  7508.0   177.0  2156.0   
4       2555.0  1853.0  8666.0  3898.0  2909.0  9641.0  3251.0  1531.0   
...        ...     ...     ...     ...     ...     ...     ...     ...   
262139  6900.0  7772.0  1993.0  6077.0  8445.0  9980.0  6878.0   719.0   
262140  1887.0  5856.0  7018.0  9729.0  8133.0  2427.0  1941.0  2337.0   
262141   295.0  5777.0  7249.0  4352.0  4531.0  6822.0  2551.0  4963.0   
262142  9086.0  3298.0  9103.0  1697.0  3115.0  9882.0  5171.0  4197.0   
262143   945.0  2233.0  4351.0   974.0  8169.0  7791.0  9189.0  8469.0   

          col8    col9  ...  col246  col247  col248  col249  col250  col251  \
0       8212.0  7245.0  ...   80

In [19]:
# Concat
t9 = time.time()
print(pd.concat([modin_df, modin_df], axis=0))
modin_time = time.time() - t9
print("Modin wall time for completion in seconds:",modin_time)

          col0    col1    col2    col3    col4    col5    col6    col7  \
0       3403.0  7232.0  8592.0  6451.0  5741.0  9222.0  9565.0  1560.0   
1       8905.0  7830.0  1765.0  9886.0  3208.0  7536.0  9996.0  1495.0   
2       8039.0  1927.0  9661.0  7323.0   386.0  2187.0  1262.0  7503.0   
3       7142.0  9296.0  3865.0  6093.0  2408.0  7508.0   177.0  2156.0   
4       2555.0  1853.0  8666.0  3898.0  2909.0  9641.0  3251.0  1531.0   
...        ...     ...     ...     ...     ...     ...     ...     ...   
262139  6900.0  7772.0  1993.0  6077.0  8445.0  9980.0  6878.0   719.0   
262140  1887.0  5856.0  7018.0  9729.0  8133.0  2427.0  1941.0  2337.0   
262141   295.0  5777.0  7249.0  4352.0  4531.0  6822.0  2551.0  4963.0   
262142  9086.0  3298.0  9103.0  1697.0  3115.0  9882.0  5171.0  4197.0   
262143   945.0  2233.0  4351.0   974.0  8169.0  7791.0  9189.0  8469.0   

          col8    col9  ...  col246  col247  col248  col249  col250  col251  \
0       8212.0  7245.0  ...   80

In [20]:
verify_and_print_times(pandas_time, modin_time)

Modin was 14.02X faster than stock pandas!


## Modin Coverage Examples 
The Modin package supports a large variety of Pandas functions.
Here are some examples:

### Count

In [21]:
modin_df.count()

col0      262144
col1      262144
col2      262144
col3      262144
col4      262144
           ...  
col251    262144
col252    262144
col253    262144
col254    262144
col255    262144
Length: 256, dtype: int64

### Filter

In [22]:
modin_df.filter(regex='0$', axis=1)

,col0,col10,col20,col30,col40,col50,col60,col70,col80,col90,...,col160,col170,col180,col190,col200,col210,col220,col230,col240,col250
0,3403.0,4616.0,7704.0,1309.0,533.0,5918.0,1066.0,3982.0,8693.0,9135.0,...,9764.0,9695.0,8222.0,3391.0,4137.0,1512.0,4470.0,5772.0,2573.0,9975.0
1,8905.0,3973.0,6156.0,9023.0,7606.0,6136.0,8581.0,9329.0,1200.0,7279.0,...,232.0,6681.0,8928.0,6675.0,565.0,8397.0,6054.0,5977.0,7343.0,4443.0
2,8039.0,5345.0,1921.0,7332.0,7075.0,5302.0,7376.0,5368.0,8819.0,2316.0,...,7302.0,4228.0,1768.0,3363.0,9915.0,9898.0,3064.0,319.0,2827.0,3143.0
3,7142.0,7906.0,9952.0,1658.0,3352.0,817.0,445.0,8872.0,3302.0,8105.0,...,5500.0,8455.0,487.0,1208.0,8312.0,6810.0,3683.0,2309.0,672.0,5378.0
4,2555.0,6672.0,1876.0,6247.0,2482.0,446.0,1350.0,8505.0,4573.0,5633.0,...,1018.0,2086.0,2241.0,8624.0,6785.0,7352.0,9814.0,8423.0,879.0,4877.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
262139,6900.0,7623.0,576.0,2880.0,5647.0,1074.0,3443.0,221.0,2490.0,1920.0,...,1288.0,3388.0,8345.0,3220.0,1457.0,3109.0,4719.0,4356.0,391.0,8459.0
262140,1887.0,5494.0,8244.0,7513.0,6965.0,2010.0,5322.0,7539.0,6140.0,7384.0,...,8716.0,849.0,6955.0,8252.0,5572.0,856.0,2081.0,1625.0,9668.0,4515.0
262141,295.0,6680.0,1807.0,6066.0,9987.0,1318.0,4329.0,9410.0,6364.0,2153.0,...,3709.0,1709.0,8957.0,9844.0,4945.0,1090.0,5043.0,2545.0,3840.0,3713.0
262142,9086.0,7085.0,8772.0,9443.0,7096.0,2989.0,2197.0,2110.0,8478.0,8945.0,...,3352.0,591.0,7881.0,5446.0,947.0,7016.0,214.0,5311.0,4464.0,1164.0


### iloc

In [23]:
modin_df.iloc[0]
modin_df.iloc[-1]
modin_df.iloc[:,0]
modin_df.iloc[:,-1]

0         3417.0
1         8320.0
2         1575.0
3         8022.0
4         9271.0
           ...  
262139    7670.0
262140    6610.0
262141    9992.0
262142    9200.0
262143    4120.0
Name: col255, Length: 262144, dtype: float64

## Series

In [24]:
s = pd.Series(np.random.randn(5), index=['a', 'b', 'c', 'd', 'e'])

### DataFrame to NumPy Array

In [25]:
modin_df.to_numpy()

array([[3403., 7232., 8592., ..., 1690., 3314., 3417.],
       [8905., 7830., 1765., ..., 4717.,  509., 8320.],
       [8039., 1927., 9661., ..., 6071., 1431., 1575.],
       ...,
       [ 295., 5777., 7249., ...,  690., 4931., 9992.],
       [9086., 3298., 9103., ..., 2239., 3584., 9200.],
       [ 945., 2233., 4351., ..., 4111., 5329., 4120.]])

### Series to NumPy Array

In [26]:
ser = pd.Series(pd.date_range('2000', periods=2, tz="CET"))
ser.to_numpy(dtype=object)
ser.to_numpy(dtype="datetime64[ns]")

array([Timestamp('2000-01-01 00:00:00+0100', tz='CET'),
       Timestamp('2000-01-02 00:00:00+0100', tz='CET')], dtype=object)

### Set Options

In [27]:
pd.set_option('compute.use_bottleneck', False)
pd.set_option('compute.use_numexpr', False)

### Unique Function for Series

In [28]:
pd.unique(pd.Series([2, 1, 3, 3]))

array([2, 1, 3])

In [29]:
print("[CODE_SAMPLE_COMPLETED_SUCCESFULLY]")

[CODE_SAMPLE_COMPLETED_SUCCESFULLY]
